In [ ]:
import numpy as np
import marshal, json
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import matplotlib.patches as patches
import matplotlib
import json, math
import imageio.v2 as imageio
from svgpath2mpl import parse_path
import scipy.stats
from IPython.display import set_matplotlib_formats
from IPython.display import Image
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
plt.rcParams.update({
    'font.family': 'serif',
    "font.serif": 'Times',
    'font.size': 12,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'text.latex.preamble': [r'\usepackage{amsmath}']
})

########## PARAMS ##########
num_runs = 10
diss_rate_target = 0.99
aoi_target = 0.01
v = 26
frame_count = 5

density_low = 4.59598735
density_high = 11.95521146

q_list = [100, 120, 140, 160, 180, 200]
p_list = [100, 90, 80, 70, 60, 50]
i_list = [60, 120, 180, 240, 300, 360, 420]

nn = np.arange(100, 500, 20)

########## UTIL FUNCTIONS ##########

def calculate_confidence_interval(data, confidence=.95):
    n = len(data)
    m = np.mean(data)
    std_dev = scipy.stats.sem(data)
    h = std_dev * scipy.stats.t.ppf((1 + confidence) / 2, n - 1)
    return [m, h]

def cis_overlap(l_1, h_1, l_2, h_2):
    return (l_1 >= l_2 and l_1 <= h_2) or (h_1 >= l_2 and h_1 <= h_2)

def is_successful_config(diss_rate, excess_probability, config):
    return diss_rate[config] >= diss_rate_target and excess_probability[config] <= aoi_target


def get_best_config(diss_rate, excess_probability, diss_rate_ci):
    meets_reliability_target = len(diss_rate[diss_rate >= diss_rate_target]) > 0
    if meets_reliability_target:
        candidates = diss_rate >= diss_rate_target
        candidates = (1 - excess_probability) * candidates
        return np.unravel_index(np.argmax(candidates), candidates.shape)
    else:
        best = np.unravel_index(np.argmax(diss_rate), diss_rate.shape)
        return best

def get_best_excess_probability_config(diss_rate, excess_probability, diss_rate_ci):
    meets_reliability_target = len(diss_rate[diss_rate >= diss_rate_target]) > 0
    meets_delay_target = len(excess_probability[excess_probability <= aoi_target]) > 0

    meets_both_targets_candidates = ((excess_probability <= aoi_target) & (diss_rate >= diss_rate_target))

    meets_both_targets = np.sum(meets_both_targets_candidates) > 0

    if meets_both_targets:
        candidates = excess_probability * meets_both_targets_candidates
        best = np.unravel_index(np.argmin(candidates), candidates.shape)
        return best
    if meets_delay_target:
        candidates = excess_probability <= aoi_target
        candidates = diss_rate * candidates
        return np.unravel_index(np.argmax(candidates), candidates.shape)
    else:
        best = np.unravel_index(np.argmin(excess_probability), excess_probability.shape)
        return best
    # best = np.unravel_index(np.argmin(excess_probability), excess_probability.shape)
    # return best

def get_rdf_config_performance(n,i,q):
    diss_rate = []
    excess_probability = []

    errors = 0
    for r in range(num_runs):
        try:
            with open(f'../res/v{v}_parsed/summary_rdf_n{n}_i{i}_q{q}_r{r}.json', 'r') as f:
                data = json.load(f)
                diss_rate.append(data['avg_dissemination_rate'])
                excess_probability.append(data['excess_probability_1_R'])
        except:
            errors += 1

    if errors == num_runs:
        return (0, 1, 0 ,0)

    (avg_diss_rate, diss_rate_ci) = calculate_confidence_interval(diss_rate)
    (excess_probability, excess_probability_ci) = calculate_confidence_interval(excess_probability)
    return (avg_diss_rate, excess_probability, diss_rate_ci, excess_probability_ci)


def get_sf_config_performance(n,i,p):
    diss_rate = []
    excess_probability = []
    
    errors = 0
    for r in range(num_runs):
        try:
            with open(f'../res/v{v}_parsed/summary_sf_n{n}_i{i}_p{p}_r{r}.json', 'r') as f:
                data = json.load(f)
                diss_rate.append(data['avg_dissemination_rate'])
                excess_probability.append(data['excess_probability_1_R'])
        except:
            errors += 1

    if errors == num_runs:
        return (0, 1, 0, 0)
        
    (avg_diss_rate, diss_rate_ci) = calculate_confidence_interval(diss_rate)
    (excess_probability, excess_probability_ci) = calculate_confidence_interval(excess_probability)
    return (avg_diss_rate, excess_probability, diss_rate_ci, excess_probability_ci)

In [ ]:
def get_rdf_performance(num_nodes):
    reliability_target = np.zeros((len(q_list), len(i_list)))
    delay_target = np.zeros((len(q_list), len(i_list)))
    reliability_target_ci = np.zeros((len(q_list), len(i_list)))
    delay_target_ci = np.zeros((len(q_list), len(i_list)))

    for x, q in enumerate(q_list):
        for y, i in enumerate(i_list):
            (diss_rate, excess_probability, diss_rate_ci, excess_probability_ci) = get_rdf_config_performance(num_nodes, i, q)
            reliability_target[x][y] = diss_rate
            delay_target[x][y] = excess_probability
            reliability_target_ci[x][y] = diss_rate_ci
            delay_target_ci[x][y] = excess_probability_ci

    return (reliability_target, delay_target, reliability_target_ci, delay_target_ci)

def get_sf_performance(num_nodes):
    reliability_target = np.zeros((len(q_list), len(i_list)))
    delay_target = np.zeros((len(q_list), len(i_list)))
    reliability_target_ci = np.zeros((len(q_list), len(i_list)))
    delay_target_ci = np.zeros((len(q_list), len(i_list)))

    for x, p in enumerate(p_list):
        for y, i in enumerate(i_list):
            (diss_rate, excess_probability, diss_rate_ci, excess_probability_ci) = get_sf_config_performance(num_nodes, i, p)
            reliability_target[x][y] = diss_rate
            delay_target[x][y] = excess_probability
            reliability_target_ci[x][y] = diss_rate_ci
            delay_target_ci[x][y] = excess_probability_ci

    return (reliability_target, delay_target, reliability_target_ci, delay_target_ci)
    

In [ ]:
sf_ep = []
sf_ep_alt = []
sf_dr = []
sf_dr_alt = []
sf_ep_ci = []
sf_ep_alt_ci = []
sf_dr_ci = []
sf_dr_alt_ci = []
rdf_ep = []
rdf_ep_alt = []
rdf_dr = []
rdf_dr_alt = []
rdf_ep_ci = []
rdf_ep_alt_ci = []
rdf_dr_ci = []
rdf_dr_alt_ci = []

rdf_trajectory = []
rdf_trajectory_alt = []
sf_trajectory = []
sf_trajectory_alt = []


for n in nn:
    (reliability_target, delay_target, reliability_target_ci, delay_target_ci) = get_sf_performance(n)
    best = get_best_config(reliability_target, delay_target, reliability_target_ci)
    if len(sf_trajectory) >= 1:
        last_config = sf_trajectory[-1]
        rtl = reliability_target[last_config]
        rtl_ci = reliability_target_ci[last_config]
        rtb = reliability_target[best]
        rtb_ci = reliability_target_ci[best]
        dtl = delay_target[last_config]
        dtl_ci = delay_target_ci[last_config]
        dtb = delay_target[best]
        dtb_ci = delay_target_ci[best]

        rtl_met = rtl >= diss_rate_target
        rtb_met = rtb >= diss_rate_target
        dtl_met = dtl <= aoi_target
        dtb_met = dtl <= aoi_target
        l_met = rtl_met and dtl_met
        b_met = dtb_met and rtb_met
        # if b_met and not l_met:
        #     best = best
        # elif cis_overlap(rtl-rtl_ci, rtl+rtl_ci, rtb-rtb_ci, rtb+rtb_ci) and cis_overlap(dtl-dtl_ci, dtl+dtl_ci, dtb-dtb_ci, dtb+dtb_ci):
        #     best = last_config

    sf_trajectory.append(best)
    sf_ep.append(delay_target[best] * 100)
    sf_dr.append(reliability_target[best] * 100)
    sf_ep_ci.append(delay_target_ci[best] * 100)
    sf_dr_ci.append(reliability_target_ci[best] * 100)
    ## Alternative best
    best = get_best_excess_probability_config(reliability_target, delay_target, reliability_target_ci)
    if len(sf_trajectory_alt) >= 1:
        last_config = sf_trajectory_alt[-1]
        rtl = reliability_target[last_config]
        rtl_ci = reliability_target_ci[last_config]
        rtb = reliability_target[best]
        rtb_ci = reliability_target_ci[best]
        dtl = delay_target[last_config]
        dtl_ci = delay_target_ci[last_config]
        dtb = delay_target[best]
        dtb_ci = delay_target_ci[best]

        rtl_met = rtl >= diss_rate_target
        rtb_met = rtb >= diss_rate_target
        dtl_met = dtl <= aoi_target
        dtb_met = dtl <= aoi_target
        l_met = rtl_met and dtl_met
        b_met = dtb_met and rtb_met
        if b_met and not l_met:
            best = best
        # elif cis_overlap(rtl-rtl_ci, rtl+rtl_ci, rtb-rtb_ci, rtb+rtb_ci) and cis_overlap(dtl-dtl_ci, dtl+dtl_ci, dtb-dtb_ci, dtb+dtb_ci):
        #     best = last_config

    sf_trajectory_alt.append(best)
    sf_ep_alt.append(delay_target[best] * 100)
    sf_dr_alt.append(reliability_target[best] * 100)
    sf_ep_alt_ci.append(delay_target_ci[best] * 100)
    sf_dr_alt_ci.append(reliability_target_ci[best] * 100)

    (reliability_target, delay_target, reliability_target_ci, delay_target_ci) = get_rdf_performance(n)
    best = get_best_config(reliability_target, delay_target, reliability_target_ci)
    if len(rdf_trajectory) >= 1:
        last_config = rdf_trajectory[-1]
        rtl = reliability_target[last_config]
        rtl_ci = reliability_target_ci[last_config]
        rtb = reliability_target[best]
        rtb_ci = reliability_target_ci[best]
        dtl = delay_target[last_config]
        dtl_ci = delay_target_ci[last_config]
        dtb = delay_target[best]
        dtb_ci = delay_target_ci[best]

        rtl_met = rtl >= diss_rate_target
        rtb_met = rtb >= diss_rate_target
        dtl_met = dtl <= aoi_target
        dtb_met = dtl <= aoi_target
        l_met = rtl_met and dtl_met
        b_met = dtb_met and rtb_met
        # if b_met and not l_met:
        #     best = best
        # elif cis_overlap(rtl-rtl_ci, rtl+rtl_ci, rtb-rtb_ci, rtb+rtb_ci) and cis_overlap(dtl-dtl_ci, dtl+dtl_ci, dtb-dtb_ci, dtb+dtb_ci):
        #     best = last_config

    rdf_trajectory.append(best)
    rdf_ep.append(delay_target[best] * 100)
    rdf_dr.append(reliability_target[best] * 100)
    rdf_ep_ci.append(delay_target_ci[best] * 100)
    rdf_dr_ci.append(reliability_target_ci[best] * 100)
    ## Alternative best
    best = get_best_excess_probability_config(reliability_target, delay_target, reliability_target_ci)
    if len(rdf_trajectory_alt) >= 1:
        last_config = rdf_trajectory_alt[-1]
        rtl = reliability_target[last_config]
        rtl_ci = reliability_target_ci[last_config]
        rtb = reliability_target[best]
        rtb_ci = reliability_target_ci[best]
        dtl = delay_target[last_config]
        dtl_ci = delay_target_ci[last_config]
        dtb = delay_target[best]
        dtb_ci = delay_target_ci[best]
        
        rtl_met = rtl >= diss_rate_target
        rtb_met = rtb >= diss_rate_target
        dtl_met = dtl <= aoi_target
        dtb_met = dtl <= aoi_target
        l_met = rtl_met and dtl_met
        b_met = dtb_met and rtb_met
        if b_met and not l_met:
            best = best
        # elif cis_overlap(rtl-rtl_ci, rtl+rtl_ci, rtb-rtb_ci, rtb+rtb_ci) and cis_overlap(dtl-dtl_ci, dtl+dtl_ci, dtb-dtb_ci, dtb+dtb_ci):
        #     best = last_config

    rdf_trajectory_alt.append(best)
    rdf_ep_alt.append(delay_target[best] * 100)
    rdf_dr_alt.append(reliability_target[best] * 100)
    rdf_ep_alt_ci.append(delay_target_ci[best] * 100)
    rdf_dr_alt_ci.append(reliability_target_ci[best] * 100)


sf_ep = np.array(sf_ep)
sf_ep_alt = np.array(sf_ep_alt)
sf_dr = np.array(sf_dr)
sf_dr_alt = np.array(sf_dr_alt)
sf_ep_ci = np.array(sf_ep_ci)
sf_ep_alt_ci = np.array(sf_ep_alt_ci)
sf_dr_ci = np.array(sf_dr_ci)
sf_dr_alt_ci = np.array(sf_dr_alt_ci)
rdf_ep = np.array(rdf_ep)
rdf_ep_alt = np.array(rdf_ep_alt)
rdf_dr = np.array(rdf_dr)
rdf_dr_alt = np.array(rdf_dr_alt)
rdf_ep_ci = np.array(rdf_ep_ci)
rdf_ep_alt_ci = np.array(rdf_ep_alt_ci)
rdf_dr_ci = np.array(rdf_dr_ci)
rdf_dr_alt_ci = np.array(rdf_dr_alt_ci)


In [ ]:
## Plot KPI comparison
ep_max = 10
width = 13

rdf_color = '#7eb0d5'
sf_color = '#fd7f6f'

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot([40, 540], [0,0], color='#333', lw=1, clip_on=False)
ax1.plot([80, 500], [1,1], color='#333', lw=1)
ax1.text(85, 1.1, '$P_{E,500} = 1$', ha="left", va="bottom", color="#333", fontsize=12)

ax2.plot([80, 500], [99, 99], color='#333', lw=1)
ax2.text(495, 101, '$P_{D} = 99$', ha="right", va="bottom", color="#333", fontsize=12)

connected_at =1.44 * (6 / 0.509)**2

ax1.fill_between([density_low*36, connected_at], [-1,-1], [ep_max * 1.07, ep_max * 1.07], facecolor='#ebebeb', interpolate=True, clip_on=False, hatch='\\\\', ec='#fff', lw=0)
ax1.fill_between([connected_at, density_high* 36], [-1,-1], [ep_max * 1.07, ep_max * 1.07], facecolor='#ebebeb', interpolate=True, clip_on=False)
ax1.text(density_high * 36, ep_max * 1.12, "DOOLE REGION",  ha="right", va="top", color='#333', fontsize=10)

ax1.set_xlabel('\# Nodes')
ax1.set_ylabel('Excess Probability $P_{E,500}$ [\%]')
## RDF
ax1.plot(nn, rdf_ep, '.-', color=rdf_color)
ax1.fill_between(nn, rdf_ep-rdf_ep_ci, rdf_ep+rdf_ep_ci, facecolor=rdf_color, interpolate=True, alpha=0.2)
## RDF ALT
# ax1.plot(nn, rdf_ep_alt, '.--', color=rdf_color)
# ax1.fill_between(nn, rdf_ep_alt-rdf_ep_alt_ci, rdf_ep_alt+rdf_ep_alt_ci, facecolor=rdf_color, interpolate=True, alpha=0.2)

## SF
ax1.plot(nn, sf_ep, '.-', color=sf_color)
ax1.fill_between(nn, sf_ep - sf_ep_ci, sf_ep + sf_ep_ci, facecolor=sf_color, interpolate=True, alpha=0.2)
## SF ALT
# ax1.plot(nn, sf_ep_alt, '.--', color=sf_color)
# ax1.fill_between(nn, sf_ep_alt - sf_ep_alt_ci, sf_ep_alt + sf_ep_alt_ci, facecolor=sf_color, interpolate=True, alpha=0.2)

ax1.tick_params(axis='y')
ax1.set_ylim(-1, ep_max * 1.0)
ax1.set_xlim(80, 500)
ax1.set_yticks([-0.5] + list(np.arange(1, ep_max + 1, 1)))
ax1.set_yticklabels(['KPIs'] + list(range(1, ep_max + 1)))

ax2.set_ylabel('Avg Dissemination Rate $P_D$ [\%]')

ax2.plot(nn, rdf_dr, '1-', color=rdf_color)
ax2.fill_between(nn, rdf_dr - rdf_dr_ci, rdf_dr + rdf_dr_ci, facecolor=rdf_color, interpolate=True, alpha=0.2)

## RDF ALT
# ax2.plot(nn, rdf_dr_alt, '1--', color=rdf_color)
# ax2.fill_between(nn, rdf_dr_alt - rdf_dr_alt_ci, rdf_dr_alt + rdf_dr_alt_ci, facecolor=rdf_color, interpolate=True, alpha=0.2)

ax2.plot(nn, sf_dr, '1-', color=sf_color)
ax2.fill_between(nn, sf_dr - sf_dr_ci, sf_dr + sf_dr_ci, facecolor=sf_color, interpolate=True, alpha=0.2)

for i in range(len(nn)):
    n = nn[i]
    ep_target_met = rdf_ep[i] <= aoi_target * 100
    dr_target_met = rdf_dr[i] >= diss_rate_target * 100
    txt = ''

    if ep_target_met and dr_target_met:
        t1 = matplotlib.patches.Ellipse((n,-0.5), width=width, height=width / 420 * 17, fill=True, fc=rdf_color, lw=0, ec= '#fff', clip_on=False)
        ax1.add_patch(t1)
        ax1.plot([n-2.3, n-0.6, n+3],[-0.5, -0.58, -0.4], color='#fff', lw=1)

    ep_target_met = sf_ep[i] <= aoi_target * 100
    dr_target_met = sf_dr[i] >= diss_rate_target * 100

    if ep_target_met and dr_target_met:
        mask = plt.Polygon([[n -10, -1], [n + 10, 0], [n+10, -1]], fill=False, lw=0)
        ax1.add_patch(mask)
        t1 = matplotlib.patches.Ellipse((n,-0.5), width=width, height=width / 420 * 17, fill=True, fc=sf_color, lw=0, clip_path=mask)
        ax1.add_patch(t1)
        ax1.plot([n-2.3, n-0.6, n+3],[-0.5, -0.58, -0.4], color='#fff', lw=1)

ax2.tick_params(axis='y')
ax2.set_ylim(-10, 100)
ax2.set_xlim(80, 500)
ax2.set_yticks(np.arange(10, 110, 10))
ax1.spines.top.set_visible(False)
ax2.spines.top.set_visible(False)

plt.plot([-1],[0], label='CBF / RDF', color=rdf_color)
plt.plot([-1],[0], label='PF / SF', color=sf_color)
plt.plot([-1],[0], '1', label='$P_D$', color='#333')
plt.plot([-1],[0], '.',label='$P_{E,500}$', color='#333')
leg = plt.legend(fancybox=False, loc=7, bbox_to_anchor=(1, 0.78), framealpha=1.0)
leg.get_frame().set_edgecolor('#333')

fig.savefig(f"../figures/best_config_performance_v{v}.pdf", dpi=500, bbox_inches='tight', pad_inches=0.01)
fig.savefig(f"../figures/best_config_performance_v{v}.png", dpi=500, bbox_inches='tight', pad_inches=0.01)

plt.show()
    



In [ ]:
def list_to_text(stay_list):
    if len(stay_list) <= 1:
        return f'${stay_list[0]}$'
        return ', '.join([str(x) for x in stay_list])
    else:
        return f'${stay_list[0]} - {stay_list[-1]}$'
    if len(stay_list) >= 4:
        half_len = int(len(stay_list) / 2)
        first_half = stay_list[:half_len]
        second_half = stay_list[half_len:]
        return ', '.join([str(x) for x in first_half]) + '\n' + ', '.join([str(x) for x in second_half])

## Plot config trajectory
fig, ax = plt.subplots()

for x in range(len(i_list) +1):
    plt.plot([x - 0.5, x - 0.5],[-0.5, len(p_list)- 0.5], color='#dadada', clip_on=False, lw=1)


for y in range(len(p_list) +1):
    plt.plot([-0.5, len(i_list)- 0.5],[y- 0.5, y -0.5], color='#dadada', clip_on=False, lw=1)
    

start = 0.3
step = start / 5
l1 = list(zip(start-np.arange(0, 2* start + step, step), 0*(np.arange(0, 2* start + step, step) - start)))
l2 = reversed(l1)
offset = l1 # [val for pair in zip(l1, l2) for val in pair]
direction = -1
idx = 1
stay_list = [100]

dir_list = [1, -1, 1, 1, -1, 1, -1, 1, 1]
ha_list = ['left', 'right', 'right', 'left', 'left', 'left', 'right', 'right', 'right', 'right']
va_list = ['bottom', 'bottom', 'bottom', 'bottom', 'bottom', 'bottom', 'bottom', 'bottom', 'bottom', 'bottom']


for i in range(1, len(sf_trajectory)):
    x_start = sf_trajectory[i-1][1] + offset[idx-1][0]
    x_end = sf_trajectory[i][1] +  offset[idx][0]
    y_start = 5 - sf_trajectory[i-1][0] + offset[idx-1][1]
    y_end = 5 - sf_trajectory[i][0] + offset[idx][1]
    dist = math.sqrt((x_end-x_start) ** 2 + (y_end - y_start) ** 2)
    if dist >= 0.5:
        rad = 0.3 * dir_list[idx -1]
        arrow = patches.FancyArrowPatch((x_start, y_start), (x_end, y_end), connectionstyle=f"arc3,rad={rad}", arrowstyle="Simple, tail_width=0, head_width=4, head_length=8", color=sf_color, zorder=2)
        ax.add_patch(arrow)
        ax.plot([x_end],[y_end], '.', color=sf_color)
        if idx == 1:
            ax.plot([x_start],[y_start], '.', color=sf_color)
        ax.text(x_start, y_start, list_to_text(stay_list), ha=ha_list[idx-1], va=va_list[idx-1], color=sf_color, linespacing=1.5)
        idx += 1
        stay_list = [nn[i]]
    else:
        stay_list.append(nn[i])

ax.text(sf_trajectory[-1][1] + offset[idx][0], 5 - sf_trajectory[-1][0] + offset[idx][1], list_to_text(stay_list), ha='right', va="top", color=sf_color, linespacing=1.5)

stay_list=[100]
dir_list = [1, 1, 1, 1, 1, 1, 1, 1]
ha_list = ['center', 'center', 'left', 'right', 'center', 'center', 'center', 'center']
va_list = ['top', 'bottom', 'bottom', 'bottom', 'top', 'top', 'top', 'top']
idx = 1


for i in range(1, len(rdf_trajectory)):
    x_start = rdf_trajectory[i-1][1] + offset[idx-1][0]
    x_end = rdf_trajectory[i][1] +  offset[idx][0]
    y_start = 5 - rdf_trajectory[i-1][0] + offset[idx-1][1]
    y_end = 5 - rdf_trajectory[i][0] + offset[idx][1]
    dist = math.sqrt((x_end-x_start) ** 2 + (y_end - y_start) ** 2)
    if dist >= 0.5:
        rad = 0.3 * dir_list[idx -1]
        arrow = patches.FancyArrowPatch((x_start, y_start), (x_end, y_end), connectionstyle=f"arc3,rad={rad}", arrowstyle="Simple, tail_width=0, head_width=4, head_length=8", color=rdf_color, zorder=2)
        ax.add_patch(arrow)
        ax.plot([x_end],[y_end], '.-', color=rdf_color)
        if idx == 1:
            ax.plot([x_start],[y_start], '.-', color=rdf_color)
        ax.text(x_start, y_start, list_to_text(stay_list), ha=ha_list[idx-1], va=va_list[idx-1], color=rdf_color)
        idx += 1
        stay_list = [nn[i]]
    else:
        stay_list.append(nn[i])

ax.text(rdf_trajectory[-1][1], 5 - rdf_trajectory[-1][0], list_to_text(stay_list), ha='center', va="top", color=rdf_color, linespacing=1.5)



y_labels = [f'${p_list[i] / 100} / {q_list[i] / 100}$' for i in range(len(q_list))]
fig.tight_layout()
plt.axis([-0.5, 6.5, -0.5, 5.5])
ax.set_yticks(np.arange(len(p_list)), labels=reversed(y_labels))
ax.set_xticks(np.arange(len(i_list)), labels=i_list)

ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)
# ax.spines.left.set_visible(False)
# ax.spines.bottom.set_visible(False)

ax.set_xlabel('Send Interval [ms]')
ax.set_ylabel('Forwarding Probability $p$ /  Decay Factor $q$')

ax.plot([-10],[-10], '.-', color=rdf_color, label='CBF / RDF')
ax.plot([-10],[-10], '.-', color=sf_color, label='PF / SF')
leg = ax.legend(fancybox=False, framealpha=1.0)
leg.get_frame().set_edgecolor('#333')
#plt.title('Configuration Trajectory')

ax.set_aspect('equal')
fig.savefig(f"../figures/config_trajectory_v{v}.pdf", dpi=500, bbox_inches='tight', pad_inches=0.01)
fig.savefig(f"../figures/config_trajectory_v{v}.png", dpi=500, bbox_inches='tight', pad_inches=0.01)
plt.show()

print(sf_trajectory)
